In [3]:
import chemfp
# from chemfp.bitops import 
import pandas as pd
from rdkit.Chem import PandasTools, Draw
from rdkit import DataStructs
from rdkit.ML.Cluster import Butina
from rdkit.Chem import rdMolDescriptors as rdmd
from rdkit.Chem import Descriptors
import seaborn as sns
from IPython.display import HTML
from tqdm import tqdm
from sklearn.metrics import adjusted_rand_score as ari
from sklearn.metrics import silhouette_scorepKi_threshold = 4

In [2]:
def butina_cluster(mol_list,cutoff=0.35):
    fp_list = [rdmd.GetMorganFingerprintAsBitVect(m, 2, nBits=1024) for m in mol_list]
    dists = []
    nfps = len(fp_list)
    for i in tqdm(range(1, nfps)):
        sims = DataStructs.BulkTanimotoSimilarity(fp_list[i],fp_list[:i])
        dists.extend([1-x for x in sims])
    mol_clusters = Butina.ClusterData(dists,nfps,cutoff,isDistData=True)
    cluster_id_list = [0]*nfps
    print(len(cluster_id_list))
    for idx,cluster in enumerate(mol_clusters,1):
        for member in cluster:
            cluster_id_list[member] = idx
    return cluster_id_list, mol_clusters

In [3]:
df = pd.read_csv("BL_Sets_data_rdkitblog.csv", sep=",")
df = df[df["pKi"]>pKi_threshold]
PandasTools.AddMoleculeColumnToFrame(df,smilesCol="smiles")
from time import time
st = time()
y = butina_cluster(df.ROMol.values)
print("Time: ", time()-st)

100%|██████████| 91662/91662 [17:28<00:00, 87.46it/s]  


Time:  1856.2847983837128


In [52]:
data.shape

(91662, 1024)

In [1]:
import numpy as np
import pandas as pd
from rdkit import Chem, DataStructs
from rdkit.Chem import rdFingerprintGenerator
from classix_m import CLASSIX_M
from classix_t import CLASSIX_T
# Setting up the Hyperparameters

regenerateFromScratch = False   # generate Morgan Fingerprints can take a while, so we save
pKi_threshold = 4
tanimoto_radius = 0.33
manhattan_radius = 1.3
mergeScale = 1.33
minPts = 50

# Loading the Data
if regenerateFromScratch:
    # Data should be generated from scratch if the pKi threshold is changed.
    
    fp_gen = rdFingerprintGenerator.GetMorganGenerator(
        radius=2,  # atom radius to consider for hashing
        fpSize=1024  # bit-length of vector (smaller will lead to more collissions during hashing) 
    )
    df = pd.read_csv("BL_Sets_data_rdkitblog.csv", sep=",")
    df["mol"] = df["smiles"].apply(Chem.MolFromSmiles)
    df = df[df["pKi"]>pKi_threshold]
    data = np.vstack(df["mol"].apply(fp_gen.GetFingerprintAsNumPy))
    np.save('BL_Sets.npy', data, allow_pickle=True)
else:
    df = pd.read_csv("BL_Sets_data_rdkitblog.csv", sep=",")
    df = df[df["pKi"]>pKi_threshold]
    data = np.load('BL_Sets.npy')

cluster_membership = df["target_chembl_id"].astype("category").cat.codes
true_labels = np.array(cluster_membership)

# Initialise the models
clx_t = CLASSIX_T(radius=tanimoto_radius, minPts=minPts, mergeScale=mergeScale)
clx_m = CLASSIX_M(radius=1, minPts=100, mergeScale=1.3)

data = data.astype(np.int32)
# clx_m.fit(data)
# Run Tanimoto Clustering
clx_t.fit(data, r=tanimoto_radius, mergeScale=mergeScale)


OWN AGGREGATION


100%|██████████| 90498/90498 [00:42<00:00, 2136.06it/s] 


time for ips: 34.10639977455139
time for search: 1.7810029983520508
time for conversion: 2.0306832790374756
time for loop: 3.7348947525024414
nr_dist: 588244445


  0%|          | 0/16385 [00:00<?, ?it/s]

[0]
[1]
[2]
[3]
[4]
[5]
[6]
[7]
[8]
[9]
[10]
[11]
[12]
[13]
[14]
[15]
[16]
[17]
[18 30]
[19]
[20]
[21]
[22]
[23]
[24]
[25]
[26]
[27]
[28]
[29]
[30]
[31 41]
[32]
[ 33 144]
[34]
[35]
[ 36 102]
[37]
[38]
[39]
[40]
[41]
[42 92]
[43]
[44]
[45]
[46]
[47]
[ 48 236]
[49]
[50]
[ 51 105 156]
[ 52  72 314]
[53]
[54]
[55 88]
[56]
[57 88]
[58]
[59]
[60]
[61]
[62]
[63]
[64]
[65]
[ 66 186 395 396 397 461 655]
[67 99]
[68]
[69]
[70]
[ 71  96 189]
[ 72  80  87 101 339 550]
[ 73 475]
[74 99]
[75]
[76]
[77]
[78]
[79]
[ 80  87 124 125 174 176 416 417 418 584 586 587 684 685 806 807]
[81]
[ 82 206]
[ 83 300 538]
[84]
[85]
[86 87]
[ 87 124 125]
[88]
[ 89 122 148 149 150 151 152]
[90]
[91]
[92]


...

 98%|█████████▊| 16109/16385 [00:12<00:00, 2414.40it/s]

[16102]
[16103 16104]
[16104]
[16105]
[16106 16180]
[16107]
[16108 16131]
[16109]
[16110 16210]
[16111 16340 16370]
[16112]
[16113]
[16114 16138]
[16115 16272]
[16116 16136 16216]
[16117]
[16118 16319]
[16119]
[16120 16160]
[16121]
[16122]
[16123]
[16124 16246 16257]
[16125]
[16126]
[16127 16174]
[16128]
[16129]
[16130]
[16131]
[16132]
[16133]
[16134]
[16135]
[16136 16139 16198 16216 16221 16230]
[16137 16138]
[16138]
[16139 16198 16230]
[16140]
[16141 16266]
[16142]
[16143 16144]
[16144]
[16145 16223]
[16146]
[16147]
[16148 16259]
[16149]
[16150]
[16151]
[16152]
[16153]
[16154]
[16155]
[16156]
[16157]


100%|██████████| 16385/16385 [00:12<00:00, 1320.21it/s]


[16158]
[16159]
[16160]
[16161]
[16162]
[16163 16197 16269]
[16164]
[16165]
[16166]
[16167]
[16168]
[16169]
[16170]
[16171 16301]
[16172]
[16173]
[16174]
[16175]
[16176 16307]
[16177]
[16178]
[16179 16205]
[16180]
[16181 16261]
[16182]
[16183]
[16184 16280]
[16185 16270]
[16186]
[16187 16212]
[16188]
[16189]
[16190 16191 16202 16203 16233 16245 16256 16257 16283 16284]
[16191 16202 16203 16233 16245 16256 16257 16283 16284]
[16192]
[16193]
[16194]
[16195 16368]
[16196]
[16197]
[16198 16230]
[16199]
[16200]
[16201]
[16202 16233 16245 16256 16284]
[16203 16233 16245 16284]
[16204 16223 16278]
[16205]
[16206]
[16207]
[16208]
[16209 16290 16343]
[16210]
[16211]
[16212 16243]
[16213]
[16214 16324 16378]
[16215]
[16216]
[16217]
[16218 16267]
[16219]
[16220]
[16221]
[16222 16277]
[16223]
[16224]
[16225]
[16226 16251 16289 16309 16321 16354 16356]
[16227]
[16228]
[16229]
[16230]
[16231 16272]
[16232]
[16233 16245 16246 16256 16257 16283 16284]
[16234]
[16235]
[16236 16258]
[16237]
[16238 16244

CLASSIX_T(sorting=popcount, radius=0.33, minPts=50, group_merging=tanimoto_distance, mergeScale=1.33, mergeTinyGroups=True)

In [2]:
# print(len(set(clx_m.labels)))
# print("ARI: ", ari(true_labels, clx_m.labels))
print(len(set(clx_t.labels)))
print("ARI: ", ari(true_labels, clx_t.labels))
# clx_labels = deepcopy(clx_t.labels)
# indices = np.arange(data.shape[0])
# np.random.shuffle(indices)
# print(silhouette_score(data[indices[::20]], clx_labels[indices[::20]], metric='jaccard'))
# print(silhouette_score(data[indices[::20]], true_labels[indices[::20]], metric='jaccard'))

214


NameError: name 'ari' is not defined

In [ ]:
# for i in range(len(set(clx_m.labels))):
#     print(f"Cluster {i}: ", np.sum(clx_m.labels==i))
for i in range(len(set(clx_t.labels))):
    print(f"Cluster {i}: ", np.sum(clx_t.labels==i))

In [ ]:
for i in range(len(set(true_labels))):
    print(f"Cluster {i}: ", np.sum(true_labels==i))
# for i in range(len(set(clx_t.labels))):
#     print(f"Cluster {i}: ", np.sum(clx_t.labels==i))

In [106]:
from copy import deepcopy
print(len(set(clx_t.labels)))
print("ARI: ", ari(true_labels, clx_t.labels))
s = 0
for i in range(10):
    clx_labels = deepcopy(clx_t.labels)
    indices = np.arange(data.shape[0])
    np.random.shuffle(indices)
    # data = data[indices[::10]]
    # true_labels = true_labels[indices[::10]]
    st+=silhouette_score(data[indices[::10]], true_labels[indices[::10]], metric='jaccard')
    s+=silhouette_score(data[indices[::10]], clx_labels[indices[::10]], metric='jaccard')
print(s/10)
print(st/10)

184
ARI:  0.13043280581199362


/home/research/nlagroup/.local/lib/python3.11/site-packages/sklearn/metrics/pairwise.py:2317: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/home/research/nlagroup/.local/lib/python3.11/site-packages/sklearn/metrics/pairwise.py:2317: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/home/research/nlagroup/.local/lib/python3.11/site-packages/sklearn/metrics/pairwise.py:2317: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/home/research/nlagroup/.local/lib/python3.11/site-packages/sklearn/metrics/pairwise.py:2317: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/home/research/nlagroup/.local/lib/python3.11/site-packages/sklearn/metrics/pairwise.py:2317: DataConversionWarning: Data was converted to boolean for metric ja

0.059627973088725285
173263250.08458632


In [ ]:
101
ARI:  0.01584649924321185
-0.06677582217835673

In [29]:
ari(true_labels, y[1:])


0.016839694370390967

In [88]:
ari(true_labels, db.labels_)

0.08903354398220155

In [8]:
from sklearn.cluster import DBSCAN
import time

start = time.time()
db = DBSCAN(eps=0.33, min_samples=5, metric='jaccard' ).fit(data)
end = time.time()
print("DBSCAN took", end-start, "seconds")
db_scan_time.append(end-start)
ari_score = ari(labels, db.labels_)

/home/research/nlagroup/.local/lib/python3.11/site-packages/sklearn/metrics/pairwise.py:2317: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/home/research/nlagroup/.local/lib/python3.11/site-packages/sklearn/metrics/pairwise.py:2317: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/home/research/nlagroup/.local/lib/python3.11/site-packages/sklearn/metrics/pairwise.py:2317: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/home/research/nlagroup/.local/lib/python3.11/site-packages/sklearn/metrics/pairwise.py:2317: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/home/research/nlagroup/.local/lib/python3.11/site-packages/sklearn/metrics/pairwise.py:2317: DataConversionWarning: Data was converted to boolean for metric ja

DBSCAN took 5064.842609882355 seconds


NameError: name 'db_scan_time' is not defined

In [3]:
# Run Manhattan Clustering

m_data = np.array(data, dtype=np.float32) # Need to convert data to float for Manhattan
clx_m.fit(m_data)


OWN AGGREGATION


100%|██████████| 90498/90498 [00:49<00:00, 1821.30it/s]  


  aggregation time: 49.70823526382446
  search time: 0.13016462326049805
  ips time: 49.182782888412476
  merging time: 0.57818603515625
 minPts Merging
small clusters [1 2]
CS:  [9.0493e+04 4.0000e+00 1.0000e+00]
label_sp_copy_2 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0

NameError: name 'CS' is not defined

In [6]:
clx_t.explain()

The data was clustered into 16401 groups. These were further merged into 184 clusters.


In [3]:
clx_m.minPts

50

In [72]:
from sklearn.metrics import silhouette_score

# score_data = silhouette_score(data[::100], clx_t.labels[::100], metric='jaccard')
score_data = silhouette_score(data[::100], db.labels_[::100], metric='jaccard')

/home/research/nlagroup/.local/lib/python3.11/site-packages/sklearn/metrics/pairwise.py:2317: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


In [73]:
score_data

0.05272480594421823

In [10]:
vec = np.random.rand(1024,1)
print(vec.shape)
probe = data@vec

print(len(np.unique(probe)))

(1024, 1)
51103
